In [58]:
from common_function import readConfig, appendMode, absPath, updatePipelineStatus,getLastSuccessUnix,loadBronzeInDelta,deduplicateRecords,mergeSilver
import pandas as pd
import os
from datetime import datetime
import time

In [59]:
# notebook parameters
id = "Revenues-Silver"


In [60]:
config = readConfig(id)
config

✓ Config loaded successfully: Revenues-Silver.json


{'pipeline_id': 'Revenues-Silver',
 'dependency': 'Revenues-Bronze',
 'source': {'type': 'parquet',
  'path': 'data\\01_bronze\\revenues',
  'encoding': 'utf-8'},
 'target': {'pk': ['date', 'title'],
  'order_pk': '_tf_ingestion_time',
  'format': 'parquet',
  'name': 'revenues',
  'path': 'data\\02_silver\\revenues',
  'mode': 'merge',
  'partition_by': ['date']}}

In [61]:
# get last successful run date
last_completed_date = getLastSuccessUnix(id)

In [62]:
# Read data from bronze in delta.
# Meaning - take all data which was loaded to bronze since last successful silver run

bronze_path = os.path.join(absPath(), config["source"]["path"])

df_delta = loadBronzeInDelta(
    bronze_path=bronze_path,
    partition_col='_tf_ingestion_time',
    last_success_unix=last_completed_date
)

✓ Loaded 1013459 records from C:\Users\rogoz\Documents\own_projects\futuremind-assesment\data\01_bronze\revenues


In [63]:
# deduplicate data from bronze basing on primary keys defined, order by ingestion date
df_insert = deduplicateRecords(
    df=df_delta,
    business_keys=config['target']['pk'],  # Business uniqueness
    order_by=config['target']['order_pk'],  # Keep latest ingestion
    ascending=False  # False = keep highest timestamp
)

✓ Deduplication: 1013459 → 337820 records (675639 duplicates removed)


In [64]:
# merge to silver layer based on pk

silver_path = os.path.join(absPath(), config["target"]["path"])

mergeSilver(
    df_bronze=df_insert,
    target_path=silver_path,
    primary_keys=config['target']['pk'],
    order_by=config['target']['order_pk']
)


✓ MERGE complete:
  - Updated: 337820 records
  - Inserted: 0 records
  - Total: 337820 records


In [65]:
# Usage:
updatePipelineStatus(id, status='success')

✓ Updated pipeline status: Revenues-Silver - success
